In [18]:
import math

import numpy as np
import pandas as pd
import random

random.seed(1234)
np.random.seed(1234)

import json
import glob
import itertools
from os.path import basename
from collections import *
from tqdm.notebook import tqdm
import random
from collections import Counter

from  sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import pandas as pd
import logging

from sklearn.feature_extraction.text import TfidfVectorizer
import scipy
import sklearn

from sklearn.metrics import r2_score, mean_squared_error

from sklearn.model_selection import StratifiedKFold
from sklearn.dummy import DummyRegressor


In [20]:
sklearn.__version__

'0.21.3'

In [2]:
df = pd.read_csv('/shared/0/projects/condolence/working-dir/condolence_annotator/output.csv')

In [3]:
df.head()

,Unnamed: 0,rating,target,observer,rounded,user1,user2
0,0,1.5,Thank you. I needed to hear this. The other pr...,"This sounds like the bigger issue, to an outsi...",2,1.0,2.0
1,1,2.0,"well, I don't sleep at all anyway, so that's n...",I know a few words can't undo years of mental ...,2,3.0,1.0
2,2,1.0,"Thank you. Yes, agreed on the timing, the las...","I'm so sorry, that is all really tough. We're ...",1,1.0,1.0
3,3,1.0,There have definitely been signs that things w...,Yea. I couldn't even imagine. I have unstable ...,1,1.0,1.0
4,4,1.0,I don’t know whether anybody here will remembe...,"I am sorry this is happening, that so sad and ...",1,1.0,1.0


In [4]:
if True: 
    
    rounded_to_rows = defaultdict(list)
    for i, row in df.iterrows():
        rounded_to_rows[row['rounded']].append([row['target'], row['observer'], row['rating']])    
    
    train_df = defaultdict(list)
    eval_df = defaultdict(list)
    test_df = defaultdict(list)
    for val, rows in rounded_to_rows.items():
        random.shuffle(rows)
        for i, row in enumerate(rows):
            if i % 10 == 0:
                cur_df = test_df
            elif i % 10 == 1:
                cur_df = eval_df
            else:
                cur_df = train_df
            cur_df['text_a'].append(row[0]) # target
            cur_df['text_b'].append(row[1]) # observer
            cur_df['labels'].append(row[2]) # rating
            cur_df['target'].append(row[0]) # target
            cur_df['observer'].append(row[1]) # observer
            cur_df['rating'].append(row[2]) # rating
            
    train_df = pd.DataFrame(train_df)
    eval_df = pd.DataFrame(eval_df)
    test_df = pd.DataFrame(test_df)
            
    print(len(df), len(train_df), len(eval_df), len(test_df))
        
else:
    df['text_a'] = df['target']
    df['text_b'] = df['observer']
    df['labels'] = df['rating']
    train_df, eval_df, test_df = np.split(df.sample(frac=1), [int(.8*len(df)), int(.9*len(df))])

1004 798 103 103


In [5]:
train_df.head()

,text_a,text_b,labels,target,observer,rating
0,My wife miscarried around the same time as Sav...,I'm glad to hear everything went okay for your...,2.0,My wife miscarried around the same time as Sav...,I'm glad to hear everything went okay for your...,2.0
1,I don’t think he ever loved me the same way I ...,I fell madly in love with my first boyfriend a...,1.5,I don’t think he ever loved me the same way I ...,I fell madly in love with my first boyfriend a...,1.5
2,So my 2-year wedding anniversary (and 13-year ...,"Sorry to hear that, I've been there a few time...",2.0,So my 2-year wedding anniversary (and 13-year ...,"Sorry to hear that, I've been there a few time...",2.0
3,"Exactly this. I'm currently 17, nearly 18, and...","Yes, uni is hard. You have a lot on your plate...",2.0,"Exactly this. I'm currently 17, nearly 18, and...","Yes, uni is hard. You have a lot on your plate...",2.0
4,"Man, I need to vent like every day. I wanna sa...","Of course. Losing someone so close to you, who...",2.0,"Man, I need to vent like every day. I wanna sa...","Of course. Losing someone so close to you, who...",2.0


In [6]:
count_vect = TfidfVectorizer(ngram_range=(1,2,3))

training_docs_combined = list(train_df.target) + list(train_df.observer)
X_train_counts = count_vect.fit_transform(training_docs_combined)
#print(X_train_counts.shape)
#X_train_counts[0:int(len(training_docs_combined) / 2)].shape
X_train  = scipy.sparse.hstack((X_train_counts[0:int(len(training_docs_combined) / 2)], 
                      X_train_counts[int(len (training_docs_combined) / 2):]))

In [7]:
#print(X_train.shape)
clf = RandomForestRegressor(n_estimators=100)
clf.fit(X_train, list(train_df.rating))

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [8]:
eval_docs_combined = list(eval_df.target) + list(eval_df.observer)
X_eval_counts = count_vect.transform(eval_docs_combined)
X_eval  = scipy.sparse.hstack((X_eval_counts[0:int(len(eval_docs_combined) / 2)], 
                      X_eval_counts[int(len (eval_docs_combined) / 2):]))
dev_preds = clf.predict(X_eval)

In [9]:
dev_labels = list(eval_df.rating)
print(r2_score(dev_labels, dev_preds))
print(mean_squared_error(dev_labels, dev_preds))

0.12408987215033906
0.5359985436893204


In [10]:
test_docs_combined = list(test_df.target) + list(test_df.observer)
X_test_counts = count_vect.transform(test_docs_combined)
X_eval  = scipy.sparse.hstack((X_test_counts[0:int(len(test_docs_combined) / 2)], 
                      X_test_counts[int(len (test_docs_combined) / 2):]))
test_preds = clf.predict(X_eval)

In [11]:
test_labels = list(test_df.rating)
print(r2_score(test_labels, test_preds))
print(mean_squared_error(test_labels, test_preds))

0.12843246932131236
0.49205898058252423


# Reply-only model

In [13]:
count_vect = TfidfVectorizer(ngram_range=(1,2))
X_train = count_vect.fit_transform(train_df.observer)

In [14]:
clf = RandomForestRegressor(n_estimators=100)
clf.fit(X_train, list(train_df.rating))

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [15]:
X_eval = count_vect.transform(eval_df.observer)
dev_preds = clf.predict(X_eval)
dev_labels = list(eval_df.rating)
print(r2_score(dev_labels, dev_preds))
print(mean_squared_error(dev_labels, dev_preds))

0.15906774876771423
0.5145944174757282


In [16]:
X_test = count_vect.transform(test_df.observer)
test_preds = clf.predict(X_test)
test_labels = list(test_df.rating)
print(r2_score(dev_labels, test_preds))
print(mean_squared_error(test_labels, test_preds))

0.04419109673444255
0.5166873786407767


# Dummy Regressors

In [19]:
clf = DummyRegressor(strategy='mean')
clf.fit(X_train, list(train_df.rating))
test_preds = clf.predict(X_test)
test_labels = list(test_df.rating)
print(r2_score(dev_labels, test_preds))
print(mean_squared_error(test_labels, test_preds))

-0.007755898844194142
0.5654712393254102
